# Functional Closures and Decorators

##  Closures

### Wiki says:  
In programming languages, a closure, also lexical closure or function closure, is a technique for implementing lexically scoped name binding in a language with first-class functions. Operationally, a closure is a record storing a function[a] together with an environment.[1] The environment is a mapping associating each free variable of the function (variables that are used locally, but defined in an enclosing scope) with the value or reference to which the name was bound when the closure was created.[b] Unlike a plain function, a closure allows the function to access those captured variables through the closure's copies of their values or references, even when the function is invoked outside their scope.

In [1]:
a = 'global var'
   
def outer_func():
    b = 'local to outer_func()'
    def inner_func():
        c = 'local to inner_func()'
        print(b, 'enclosing scope')
        print(a, 'global scope')
        
    inner_func()

In [2]:
outer_func()

local to outer_func() enclosing scope
global var global scope


* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which remembers a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [11]:
def make_adder(x,z):
    # print x's address
    print('id(x): %x' % id(x))
    # enclosing scope
    
    def adder(y):
        print('in adder')
        return x + y+z # Python uses LEGB to find 'x'
    
    print('id(adder): %x' % id(adder))
    return adder # we return the function adder (i.e., its address in memory) 

add39 = make_adder(39,10)
print('about to call add39')
add39(109)

id(x): 104dce5b8
id(adder): 10b098820
about to call add39
in adder


158

In [12]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
print(type(add39), repr(add39), sep='\n')

<class 'function'>
<function make_adder.<locals>.adder at 0x10b098820>


In [13]:
# all functions have a closure attribute
add39.__closure__

(<cell at 0x10bc4d460: int object at 0x104dce5b8>,
 <cell at 0x10bc4d6a0: int object at 0x104dce218>)

In [15]:
# notice that the cell object has a reference to an int object
add39.__closure__[1].cell_contents

10

In [6]:
print(make_adder.__closure__)

None


* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [ ]:
def logging(f):
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (f, args, kwargs))
        return f(*args, **kwargs)
    return wrapper

In [ ]:
logging_add39 = logging(add39)

In [ ]:
print(add39(5)) # remember that add39 just adds 39 to our argument

In [ ]:
print(logging_add39(5))

In [ ]:
logging_add39.__closure__[0].cell_contents

## Closure vs Classes?

## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (Non-Keyword Arguments) (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [16]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print(f'Running function: {func.__name__}')
        print(f'Positional arguments: {args}')
        print(f'Keyword arguments: {kwargs}')
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print(f'Result: {result}')
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

In [17]:
def add_things(a, b):
    return a + b

print('Running plain old add_things()')
print(add_things(13, 5))

Running plain old add_things()
18


In [22]:
# manual decorator assignment
cooler_add_things = document_it(add_things) 

In [23]:
print('Running cooler_add_things()')
cooler_add_things(13, 5)

Running cooler_add_things()
Running function: new_function
Positional arguments: (13, 5)
Keyword arguments: {}
Running function: add_things
Positional arguments: (13, 5)
Keyword arguments: {}
Result: 18
Result: 18


18

In [ ]:
# decorator shorthand for what we did above

# from salesforce_approved_decorator import document_it

@document_it
def add_things(a, b):
    return a + b

# add_things = document_it(add_things)

print(add_things(13, -5))

## Lab: Decorators
1. Create some function which takes an integer as its parameter
  * Create a wrapper that ensures the parameter is positive
  * use that wrapper to decorate your original function
2. Make a timer decorator that computes the elapsed time of the function wrapped by it


# Using Functools

In [32]:

from functools import wraps
 
def a_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """A wrapper function"""
        print("called before func")
        # Extend some capabilities of func
        func()
    return wrapper
 
@a_decorator
def first_function():
    """This is docstring for first function"""
    print("first function")

In [31]:
first_function()

called before func
first function
